In [1]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="35d4c2b3ff2c3777ddf7e5ffaf81a47f.KH5q4puGzSUEJuto") # 填写您自己的APIKey

In [2]:
def get_response(messages, cot = False):
    if cot:
        messages = messages + 'let\'s think step by step!'
    response = client.chat.completions.create(
        model="glm-3-turbo",  # 填写需要调用的模型名称
        # model='glm-4',
        messages=[
            {'role': 'system', 'content': 'Please output the final answer in the format of: answer is [answer], where [] is the final answer to the question and is output at the end. Please only put numbers in the output [], no other symbols.'},
            {"role": "user", "content": messages},
        ],
    )
    return response.choices[0].message

In [1]:
import pandas as pd
df = pd.read_parquet('../data/gsm8k/main/test-00000-of-00001.parquet')

In [4]:
import re
from tqdm import tqdm
zero_shot_ac = []
for id, item in tqdm(df.iterrows(), total=len(df)):
    Q = item['question']
    A = item['answer'].split('####')[1].strip()
    R = get_response(Q).content
    # 正则表达式模式，用于匹配方括号内的内容
    pattern = r'\[(.*?)\]'

    # 使用search函数从字符串末尾开始搜索
    # 为了实现从后向前搜索，可以将字符串反转，然后进行匹配
    match = re.findall(pattern, R)
    if len(match):
        match = ''.join(re.findall(r'\d+', match[-1].strip()))
        if match == A:
            zero_shot_ac.append(id)
    

100%|██████████| 1319/1319 [1:26:33<00:00,  3.94s/it] 


In [5]:
zero_shot_CoT_ac = []
for id, item in tqdm(df.iterrows(), total=len(df)):
    Q = item['question']
    A = item['answer'].split('####')[1].strip()
    R = get_response(Q, True).content
    # 正则表达式模式，用于匹配方括号内的内容
    pattern = r'\[(.*?)\]'

    # 使用search函数从字符串末尾开始搜索
    # 为了实现从后向前搜索，可以将字符串反转，然后进行匹配
    match = re.findall(pattern, R)
    if len(match):
        match = ''.join(re.findall(r'\d+', match[-1].strip()))
        if match == A:
            zero_shot_CoT_ac.append(id)
    

100%|██████████| 1319/1319 [1:57:15<00:00,  5.33s/it]  


In [6]:
len(zero_shot_ac), len(df), len(zero_shot_ac)/len(df)

(693, 1319, 0.5253980288097043)

In [8]:
len(zero_shot_CoT_ac), len(df), len(zero_shot_CoT_ac)/len(df)

(747, 1319, 0.5663381349507203)